In [1]:
import random
import matplotlib.pyplot as plt
import copy
import numpy as np
import pandas as pd

In [2]:
class Bird:
    """
    Base constructor class for birds
    """
    def __init__(self,mutation_rate=0,gene_step=0,age=0,food=0):
        self.food=food
        self.age=age
    def interact(self,opponent):
        return None    #Birds need a specific class species to interact
    def age_increase(self, ageing_rate):
        self.age += ageing_rate
    def dont_starve(self):
        death_chance = random.random()
        if self.food < death_chance:
            return False #death
        else:
            return True  #survival
    def reproduce(self):
        non_reproduction_chance = random.random()
        if (self.food - 1) < non_reproduction_chance:
            return False   # dont reproduce
        else:
            return True    # reproduce
    def mortality(self):
        death_chance = random.random()
        if self.age > death_chance:
            return False    #death
        else:
            return True     #survive
    def set_new(self):
        self.age = 0

In [3]:
#Define specific creature classes

class Dove(Bird):
    """
    Non-confrontational, shares with opponent
    """
    def __init__(self,mutation_rate=0,gene_step=0,age=0,food=0):
        self.name = 'Dove'
        self.food = food
        self.age = age
    def interact(self, opponent, payoff_dict=None):
        if opponent is None:
            self.food = 2
        elif opponent.name == 'Dove':
            self.food = 1
        elif opponent.name == 'Hawk':
            self.food = 0.5
        elif opponent.name == 'Goose':
            self.food = 1
        elif opponent.name == 'Crow':
            self.food = 0.5

class Hawk(Bird):
    """
    Aggresive, fights for food
    """
    def __init__(self,mutation_rate=0,gene_step=0, food = 0, age=0):
        self.name = 'Hawk'
        self.food = food
        self.age=age
    def interact(self, opponent, payoff_dict=None):
        if opponent is None:
            self.food = 2
        elif opponent.name == 'Dove':
            self.food = 1.5
        elif opponent.name == 'Hawk':
            self.food = 0
        elif opponent.name == 'Goose':
            self.food = 0
        elif opponent.name == 'Crow':
            self.food = 1.5

class Goose(Bird):
    """
    Nice to doves, fights with aggressors
    """
    def __init__(self,mutation_rate=0,gene_step=0, food=0, age=0):
        self.name = 'Goose'
        self.food = food
        self.age=age
    def interact(self,opponent, payoff_dict=None):
        if opponent is None:
            self.food = 2
        elif opponent.name == 'Dove':
            self.food = 1
        elif opponent.name == 'Hawk':
            self.food = 0
        elif opponent.name == 'Goose':
            self.food = 1
        elif opponent.name == 'Crow':
            self.food = 1

class Crow(Bird):
    """
    Puts up an aggresive display, reverts to dove if challenged
    """
    def __init__(self,mutation_rate=0,gene_step=0, food=0, age=0):
        self.name = 'Crow'
        self.food = food
        self.age=age
    def interact(self, opponent, payoff_dict=None):
        if opponent is None:
            self.food = 2
        elif opponent.name == 'Dove':
            self.food = 1.5
        elif opponent.name == 'Hawk':
            self.food = 0.5
        elif opponent.name == 'Goose':
            self.food = 1
        elif opponent.name == 'Crow':
            self.food = 1
            
############### Define agents for customisable payoff matrix ##############

class A(Bird):
    """
    Base class for species A
    """
    def __init__(self,mutation_rate=0,gene_step=0,age=0,food=0,largest_payoff=0):
        self.name = "A"
        self.age = age
        self.food = food
        self.largest_payoff = largest_payoff
    def interact(self, opponent, payoff_dict):
        self.largest_payoff = max(payoff_dict.values())
        if opponent is None:
            self.food = self.largest_payoff
        elif opponent.name == 'A':
            self.food = payoff_dict.get("a")
        elif opponent.name == 'B':
            self.food = payoff_dict.get("c")

class B(Bird):
    """
    Base class for species A
    """
    def __init__(self,mutation_rate=0,gene_step=0,age=0,food=0,largest_payoff=0):
        self.name = "B"
        self.age = age
        self.food = food
        self.largest_payoff = largest_payoff
    def interact(self, opponent, payoff_dict):
        self.largest_payoff = max(payoff_dict.values())
        if opponent is None:
            self.food = self.largest_payoff
        elif opponent.name == 'A':
            self.food = payoff_dict.get("b")
        elif opponent.name == 'B':
            self.food = payoff_dict.get("d")
            
################## define agents for mutation ##########################

class Mutant_0(Bird):
    """
    Base class for aggresive mutants
    """
    def __init__(self, food=0, age=0, gene_B=0, mutation_rate=0.1, gene_step=0.2):
        self.strategy = None
        self.food = food
        self.age = age
        self.gene_B = gene_B
        self.mutation_rate = mutation_rate
        self.gene_step = gene_step
        self.name = "Mutant_{}".format(self.gene_B)
    def mutate(self):
        mutation_random_chance = random.random()
        if mutation_random_chance > (1-self.mutation_rate):
            if random.choice(["up","down"]) == "up":
                self.gene_B += self.gene_step
            else:
                self.gene_B -= self.gene_step
        if self.gene_B < 0:
            self.gene_B = 0
        if self.gene_B > 1:
            self.gene_B = 1
        self.gene_B = float(format(self.gene_B, '.2f'))
        if self.gene_B == 0.00:
            self.gene_B = 0
        self.name = "Mutant_{}".format(self.gene_B)
    def pick_strategy(self):
        A_chance = random.random()
        if A_chance > self.gene_B:
            self.strategy = "A"
        else:
            self.strategy = "B"  
    def interact(self, opponent, payoff_dict):
        self.largest_payoff = max(payoff_dict.values())
        if opponent is None:
            self.food = 2
            return
        if self.strategy is None:
            self.pick_strategy()
            opponent.pick_strategy()
        if self.strategy == "A" and opponent.strategy == "A":
            self.food = payoff_dict.get("a")
        elif self.strategy == "A" and opponent.strategy == "B":
            self.food = payoff_dict.get("c")
        elif self.strategy == "B" and opponent.strategy == "A":
            self.food = payoff_dict.get("b")
        elif self.strategy == "B" and opponent.strategy == "B":
            self.food = payoff_dict.get("d")
    def set_new(self):
        self.age = 0
        self.strategy = None
        self.mutate()
        
class Mutant_Selective_0(Bird):
    """
    Base class for selected mutants
    """
    def __init__(self, food=0, age=0, gene_selective=0.5, mutation_rate=0.1, gene_step=0.2):
        self.strategy = None
        self.food = food
        self.age = age
        self.gene_selective = gene_selective
        self.mutation_rate = mutation_rate
        self.gene_step = gene_step
        self.name = "Mutant_Selective_{}".format(self.gene_selective*10)
    def mutate(self):
        mutation_random_chance = random.random()
        if mutation_random_chance > (1-self.mutation_rate):
            if random.choice(["up","down"]) == "up":
                self.gene_selective += self.gene_step
            else:
                self.gene_selective -= self.gene_step
        if self.gene_selective < 0:
            self.gene_selective = 0
        if self.gene_selective > 1:
            self.gene_selective = 1
        self.gene_selective = float(format(self.gene_selective, '.2f'))
        if self.gene_selective == 0.00:
            self.gene_selective = 0
        self.name = "Mutant_Selective_{}".format(self.gene_selective*10)
    def age_increase(self,ageing_rate=0.1):
        self.age += ageing_rate                              # no pressure
        #self.age += ageing_rate/(self.gene_selective+1)     # go high
        #self.age += ageing_rate/(2-self.gene_selective)     # go low
    def interact(self, opponent, payoff_dict=None):
        if opponent is None:
            self.food = 2
        else:
            self.food = 1
    def set_new(self):
        self.age = 0
        self.mutate()

In [4]:
def run_generation(creature_list, carrying_capacity, payoff_dict, ageing_rate):     
    #ageing rate determines age to be added per generation
    #chance of dying of old age increase with age upto 100% at age 10
    #assign creatures to resource list
    resources_used = 0
    resources = [[] for _ in range(carrying_capacity//2)]
    for creature in creature_list:
        while resources_used < carrying_capacity:
            resource_index = random.randint(0,(carrying_capacity/2)-1)
            resource = resources[resource_index]
            if len(resource) < 2:
                resources[resource_index].append(creature)
                resources_used+=1
                break
            else:
                next

    #allow creatures to interact
    #repopulate creature_list
    creature_list_food_collected = []    #creature list after foraging
    for resource in resources:
        num_creatures = len(resource)
        if num_creatures == 0:
            continue
        creature_a = resource[0]

        if num_creatures == 1:
            creature_b = None
        else: 
            creature_b = resource[1]
            creature_b.interact(creature_a, payoff_dict)
            creature_list_food_collected.append(creature_b)
        creature_a.interact(creature_b, payoff_dict)
        creature_list_food_collected.append(creature_a)
        
    #determine survival & reproduction
    creature_list = []       #creatures at end of day
    for creature in creature_list_food_collected:
        if creature.dont_starve() == False:
            next
        elif creature.mortality() == False:
            next
        else: 
            creature.age_increase(ageing_rate)
            creature_list.append(creature)
            if creature.reproduce() == True:
                new_creature = copy.deepcopy(creature)
                new_creature.set_new()
                creature_list.append(new_creature)
    for creature in creature_list:
        creature.food = 0      #new day
    random.shuffle(creature_list) 
    return(creature_list)

In [5]:
# varaible agents play Doves & Hawks by default

payoff_dict_default = {
    "a": 1,
    "b": 1.5,
    "c": 0.5,
    "d": 0
}

In [6]:
def run_simulation(starting_population=None,num_generations=50,infiltration_dict=None,infiltration_gen=20,
                   carrying_capacity=1000,payoff_dict=payoff_dict_default,ageing_rate=0.01,
                   mutation_rate=0.1,gene_step=0.2):
    creature_list = []
    generation_count=[0]
    
    #default sim
    if starting_population == None:
        starting_population = {
            "Dove" : [1],
            "Hawk" : [1],
            "Goose": [1],
            "Crow" : [1], 
            "A"    : [0], 
            "B"    : [0], 
        }

    population_dict = starting_population
    for creature in population_dict:
        creature_list.extend([globals()[creature](mutation_rate=mutation_rate,gene_step=gene_step) for i in range(population_dict[creature][0])])

    for i in range(1,num_generations):
        if infiltration_dict is not None:
            if i == infiltration_gen:
                for creature in infiltration_dict:
                    creature_list.extend([globals()[creature](mutation_rate=mutation_rate,gene_step=gene_step) for i in range(infiltration_dict[creature][0])])
                random.shuffle(creature_list)

        creature_list = run_generation(creature_list=creature_list, carrying_capacity=carrying_capacity, 
                                       payoff_dict=payoff_dict, ageing_rate=ageing_rate)
        for creature_populations in population_dict.values():
            creature_populations.append(0)
        for creature in creature_list:
            if creature.name not in population_dict:
                population_dict[creature.name] = [0] * (i+1)
            population_dict[creature.name][i] += 1
        generation_count.append(i)
    return(population_dict, generation_count)

In [12]:
def display_results(simulation_results, colors=None):
    population_dict = simulation_results[0]
    generation_count = simulation_results[1]
    plt.figure(figsize=(20,10))
    plt.stackplot(generation_count, population_dict.values(),
                  labels = population_dict.keys(),colors = colors)
    plt.gca().set_prop_cycle(None)
    for i in range(len(population_dict)):
        bird = list(population_dict.keys())[i]
        #avg = population_dict[bird][-1]
        avg = sum(population_dict[bird])/len(population_dict[bird])
        avg = [avg]*len(population_dict[bird])
        if colors is not None:
            plt.plot(generation_count,avg,linewidth=3,color = colors[i])
        else:
            plt.plot(generation_count,avg,linewidth=3)
        plt.plot(generation_count,avg,linewidth=1,color = 'white')
    plt.grid()
    plt.legend(loc = 'upper left')
    plt.show()

    total = 0
    print("Final populations:")
    for bird in population_dict:
        print(bird, population_dict.get(bird)[-1])
        total += population_dict.get(bird)[-1]
    print("\n")
    print("Total:", total)

In [13]:
def display_genes(simulation_results):
    population_dict = simulation_results[0]
    if "Mutant_Selective_0" in population_dict.keys():
        population_dict["Mutant_Selective_0"][0] = 0
    genes = {
        "gene":[],
        "pop": []
    }
    for bird in population_dict:
        if bird.startswith("Mutant_Selective_"):
            gene = bird.strip("Mutant_Selective_")
        else:
            gene = bird.strip("Mutant_")
        genes["gene"].append(float(gene))
        genes["pop"].append(population_dict[bird][-1])
    df = pd.DataFrame(genes)
    df = df.sort_values("gene")
    plt.figure(figsize=(7,7))
    plt.fill_between(df["gene"],0,df["pop"])
    plt.show()
    print(df)